In [45]:
import json
import requests
import redis
import websocket
import asyncio

In [46]:
ws=websocket.WebSocket()

In [41]:
ws.connect('ws://localhost:8000/sepsisDynamic/')

In [49]:
# ADDING A HEADER
ws.connect('ws://localhost:8000/sepsisDynamic/',
header=["User-Agent: MyProgram"])
# # ws.connect()
# # header=["User-Agent: MyProgram",
#         ...             "x-custom: header"])

In [40]:
ws.send(json.dumps({
    'type': 'echo.message',
    'data': 'This is a test message.',
    })
)
some_var = ws.recv()
print("THE VAL",some_var)

THE VAL {"type": "echo.message", "data": "This is a test message."}


In [34]:
# import asyncio
# import websocket

# async def hello():
#     uri = "ws://localhost:8000/sepsisDynamic/"
#     async with websockets.connect(uri) as websocket:
#         name = input("What's your name? ")

#         await websocket.send(name)
#         print(f"> {name}")

#         greeting = await websocket.recv()
#         print(f"< {greeting}")

# asyncio.get_event_loop().run_until_complete(hello())

<h1>Python websocket client with auth header </h1>

In [ ]:
# import asyncio
import ssl
from socket import socket

import websocket
# import websockets

def on_message(ws, message):
    print ('message received ..')
    print (message)


def on_error(ws, error):
    print ('error happened .. ')
    print (error)


def on_close(ws):
    print ("### closed ###")


def on_open(ws):

    print ('Opening Websocket connection to the server ... ')

    ## This session_key I got, need to be passed over websocket header isntad of ws.send.
    ws.send("testing message here")

websocket.enableTrace(True)

token = "........"
auth = "Authorization: Bearer " + token
ws = websocket.WebSocketApp("wss://APISERVER:8443/api/v1/namespaces/default/services/the-service:8889/proxy/websocket?token=123",
                            on_open = on_open,
                            on_message = on_message,
                            on_error = on_error,
                            on_close = on_close,
                            header = [auth]
                            )

ws.on_open = on_open

##Note: this is for --insecure flag in curl, basically to tell the client not verify the ssl certificate
ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})
# socket.setsockopt